In [1]:
import nltk
import pandas as pd

In [2]:
import nltk.classify.util #calculates accuracy
from nltk.classify import NaiveBayesClassifier #imports the classifier Naive Bayes

from nltk.corpus import stopwords #imports stopwords from nltk
from nltk.corpus import wordnet #imports wordnet(lexical database for the english language) from nltk

In [3]:
df = pd.read_csv('Data/movie_data.csv')


df.head(10)

,review,sentiment
0,"In 1974, the teenager Martha Moxley (Maggie Gr...",1
1,OK... so... I really like Kris Kristofferson a...,0
2,"***SPOILER*** Do not read this, if you think a...",0
3,hi for all the people who have seen this wonde...,1
4,"I recently bought the DVD, forgetting just how...",0
5,Leave it to Braik to put on a good show. Final...,1
6,Nathan Detroit (Frank Sinatra) is the manager ...,1
7,"To understand ""Crash Course"" in the right cont...",1
8,I've been impressed with Chavez's stance again...,1
9,This movie is directed by Renny Harlin the fin...,1


In [4]:
from unidecode import unidecode
import re
from re import sub
def clean_text(text,remove_polish_letters):
    ''' Pre process and convert texts to a list of words 
    method inspired by method from eliorc github repo: https://github.com/eliorc/Medium/blob/master/MaLSTM.ipynb'''
    text = remove_polish_letters(text)
    text = str(text)
    text = text.lower()

    # Clean the text
    text = sub(r"[^A-Za-z0-9^,!?.\/'+]", " ", text)
    text = sub(r"\+", " plus ", text)
    text = sub(r",", " ", text)
    text = sub(r"\.", " ", text)
    text = sub(r"!", " ! ", text)
    text = sub(r"\?", " ? ", text)
    text = sub(r"'", " ", text)
    text = sub(r":", " : ", text)
    text = sub(r"\s{2,}", " ", text)

    #text = text.split()

    return text

df.review = df.review.apply(lambda x: clean_text(x, unidecode))

In [5]:
df.head()

,review,sentiment
0,in 1974 the teenager martha moxley maggie grac...,1
1,ok so i really like kris kristofferson and his...,0
2,spoiler do not read this if you think about w...,0
3,hi for all the people who have seen this wonde...,1
4,i recently bought the dvd forgetting just how ...,0


In [6]:
df.sentiment.unique()

array([1, 0], dtype=int64)

In [9]:
df_array = df.values
#print(df_array)

In [95]:
X=df_array[:,0]#Reading reviews in to X
y=df_array[:,1]#Reading target positive or negative in y
y=y.astype('int')

In [46]:
#X[0:10]
#y[0:10]

In [96]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, test_size=0.2, shuffle=False)


In [48]:
#X_train.shape
#y_train.shape
#X_test.shape
#y_test.shape

In [49]:
#For BoW using Countvectorizer
from sklearn.feature_extraction.text import CountVectorizer

In [50]:
vec=CountVectorizer()

In [51]:
#fitting vectorizer on training data
vec.fit(X_train)
#vec.vocabulary_

CountVectorizer()

In [52]:
#print(vec.get_feature_names_out())
#print(len(vec.get_feature_names()))

In [53]:
#specifying upper and lower dims and removing stop words
vec=CountVectorizer(stop_words='english',min_df=.03,max_df=0.8)
vec.fit(X_train)
#vec.vocabulary_

CountVectorizer(max_df=0.8, min_df=0.03, stop_words='english')

In [54]:
#print(vec.get_feature_names())
#print(len(vec.get_feature_names()))

When we transform the matrix we get the sparse matrix which does not contain zeros and is a compressed format for easier computation. We use the command to .toarray() to get the ones and zeros back.

In [55]:
x_train_transformed=vec.transform(X_train)
x_train_transformed

<40000x492 sparse matrix of type '<class 'numpy.int64'>'
	with 1426066 stored elements in Compressed Sparse Row format>

In [56]:
#converting transformed matrix to an array
x_train=x_train_transformed.toarray()

In [57]:
pd.DataFrame(x_train_transformed.toarray(),columns=vec.get_feature_names_out()).head()

,10,able,absolutely,act,acting,action,actor,actors,actress,actually,...,worth,wouldn,writer,writing,written,wrong,year,years,yes,young
0,0,1,0,0,0,0,0,0,0,0,...,0,0,1,1,0,0,0,3,0,0
1,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,1,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [58]:
x_test_transformed=vec.transform(X_test)
x_test_transformed

<10000x492 sparse matrix of type '<class 'numpy.int64'>'
	with 358409 stored elements in Compressed Sparse Row format>

In [59]:
x_test=x_test_transformed.toarray()

In [60]:
pd.DataFrame(x_test_transformed.toarray(),columns=vec.get_feature_names_out()).head()

,10,able,absolutely,act,acting,action,actor,actors,actress,actually,...,worth,wouldn,writer,writing,written,wrong,year,years,yes,young
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,0,0,0,0,0,2,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [61]:
from sklearn.naive_bayes import MultinomialNB

In [66]:
from sklearn.naive_bayes import MultinomialNB
mnb=MultinomialNB()
#In Multinomial NB method we give the inputs as x_train and y_train. Here alpha is the term used for laplacian smoothing by default it is 1. This is used to prevent the condition of zero probability.


In [67]:
#fitting the model on training data
mnb.fit(x_train_transformed,y_train)

MultinomialNB()

In [70]:
y_pred=mnb.predict(x_test_transformed)
#predicted array
y_pred

array([1, 0, 1, ..., 0, 1, 1])

In [71]:
#Original array
y_test

array([1, 0, 1, ..., 0, 0, 1])

In [72]:
probabilty_prediction=mnb.predict_proba(x_test_transformed)
probabilty_prediction

array([[8.01569551e-02, 9.19843045e-01],
       [5.25554769e-01, 4.74445231e-01],
       [1.91474188e-03, 9.98085258e-01],
       ...,
       [9.99641238e-01, 3.58761821e-04],
       [3.80068074e-01, 6.19931926e-01],
       [3.11989047e-01, 6.88010953e-01]])

In [80]:
#The above output shows probabilities of prediction for '0' and '1'
import numpy as np
np.argmax(probabilty_prediction[1])

0

EValuating Metrics

In [81]:
from sklearn import metrics

In [82]:
metrics.accuracy_score(y_test,y_pred)

0.8128

In [83]:
confusion=metrics.confusion_matrix(y_test,y_pred)
confusion

array([[3995, 1018],
       [ 854, 4133]], dtype=int64)

In [109]:
from sklearn.metrics import classification_report,confusion_matrix
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.82      0.80      0.81      5013
           1       0.80      0.83      0.82      4987

    accuracy                           0.81     10000
   macro avg       0.81      0.81      0.81     10000
weighted avg       0.81      0.81      0.81     10000



In [110]:
#from sklearn.naive_bayes import BernoulliNB

In [111]:
#Ref:https://github.com/srbh24/NLP/blob/master/NaiveBaye'sDemo.ipynb

In [141]:
Custom_comment = "Awesome good mindblowing"
comment = clean_text(Custom_comment,unidecode)
comm_trans = vec.transform([comment])
print(mnb.predict_proba(comm_trans))
np.argmax(mnb.predict_proba(comm_trans))

[[0.48341648 0.51658352]]


1

In [142]:
Custom_comment = "This is really a bad movie, but still plot is good."
comment = clean_text(Custom_comment,unidecode)
comm_trans = vec.transform([comment])
print(mnb.predict_proba(comm_trans))
np.argmax(mnb.predict_proba(comm_trans))

[[0.88600546 0.11399454]]


0

In [143]:
Custom_comment = "This is really good movie."
comment = clean_text(Custom_comment,unidecode)
comm_trans = vec.transform([comment])
print(mnb.predict_proba(comm_trans))
np.argmax(mnb.predict_proba(comm_trans))

[[0.56380531 0.43619469]]


0

In [144]:
Custom_comment = "This is really bad movie."
comment = clean_text(Custom_comment,unidecode)
comm_trans = vec.transform([comment])
print(mnb.predict_proba(comm_trans))
np.argmax(mnb.predict_proba(comm_trans))

[[0.83731141 0.16268859]]


0

In [145]:
Custom_comment = "not worth watching,never seen this kind of movie, worst plot, and acting sucks."
comment = clean_text(Custom_comment,unidecode)
comm_trans = vec.transform([comment])
print(mnb.predict_proba(comm_trans))
np.argmax(mnb.predict_proba(comm_trans))

[[0.9768684 0.0231316]]


0

In [146]:
Custom_comment = "I enjoyed alot. The acting is next level."
comment = clean_text(Custom_comment,unidecode)
comm_trans = vec.transform([comment])
print(mnb.predict_proba(comm_trans))
np.argmax(mnb.predict_proba(comm_trans))

[[0.3936138 0.6063862]]


1

In [147]:
#https://github.com/srbh24/NLP/blob/master/NaiveBaye'sDemo.ipynb